In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re

import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MaxNLocator

%matplotlib inline

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
#for adding heavy chain
complementator = str.maketrans("ACGT", "TGCA") 

In [4]:
df_category_cancers = pd.read_csv('../data/processed/df_category_cancers_pqs.csv')

In [4]:
g4pqsconf = pd.read_csv('../data/processed/g4pqsconf.csv')
df_loop_cancer = pd.read_csv('../data/processed/df_loop_cancer_pqs.csv')
df_15_col_cancer = pd.read_csv('../data/processed/df_15_col_cancer_pqs.csv')

In [5]:
def get_norm_mutspec(df):
    
    normMut = pd.DataFrame()
    
    normMut[['A>C','A>G','A>T']] = df[['A>C','A>G','A>T']].div(df['A'], axis=0)
    normMut[['C>A','C>G','C>T']] = df[['C>A','C>G','C>T']].div(df['C'], axis=0)
    normMut[['G>A', 'G>C', 'G>T']] = df[['G>A', 'G>C', 'G>T']].div(df['G'], axis=0)
    normMut[['T>A', 'T>C', 'T>G']] = df[['T>A', 'T>C', 'T>G']].div(df['T'], axis=0)
    
    normMut = normMut.fillna(0)
    normMut = normMut.div(normMut.sum(axis=1), axis=0)
    
    normMut.columns = normMut.columns.str.translate(complementator)
    
    return normMut

In [10]:
def add_col_to_norm_mutspec(normMut:pd.DataFrame, df:pd.DataFrame):
    normMut.insert(0, 'G4_id', df['G4_id'])
    normMut.insert(1, 'side', df['side'])
    normMut = normMut.fillna(0)
    return normMut

In [13]:
norm_15_col = add_col_to_norm_mutspec(get_norm_mutspec(df_15_col_cancer).fillna(0), df_15_col_cancer)
norm_loop = add_col_to_norm_mutspec(get_norm_mutspec(df_loop_cancer).fillna(0), df_loop_cancer)

,G4_id,side,T>G,T>C,T>A,G>T,G>C,G>A,C>T,C>G,C>A,A>T,A>G,A>C
0,1,loops,0.000000,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,2,loops,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
2,3,loops,0.000000,0.000000,0.0,0.000000,0.000000,0.060870,0.730435,0.0,0.000000,0.000000,0.208696,0.0
3,4,loops,0.000000,0.068587,0.0,0.000000,0.000000,0.019204,0.816187,0.0,0.096022,0.000000,0.000000,0.0
4,5,loops,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.800000,0.0,0.000000,0.000000,0.200000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,63,loops,0.000000,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.0,0.000000,0.000000,0.500000,0.0
63,64,loops,0.136143,0.034036,0.0,0.000000,0.090762,0.000000,0.544571,0.0,0.000000,0.000000,0.194489,0.0
64,65,loops,0.020690,0.062069,0.0,0.144828,0.000000,0.289655,0.000000,0.0,0.000000,0.000000,0.482759,0.0
65,66,loops,0.000000,0.405797,0.0,0.000000,0.000000,0.289855,0.202899,0.0,0.000000,0.000000,0.101449,0.0
